<a href="https://colab.research.google.com/github/KevinTheRainmaker/MLOps/blob/main/Continuous_Adaptation_with_VertexAI_AutoML_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Continuous-Adaptation-with-VertexAI-AutoML-Pipeline
\* 해당 노트북은 박찬성(deep-diver)님의 레포지토리 'Continuous-Adaptation-with-VertexAI-AutoML-Pipeline'를 참고하여 정리한 내용입니다.

## Build Pipeline with Two Branches for GCP AutoML

1. 새로운 데이터셋 생성, 새로운 모델 학습,  새로운 Endpoint로 모델 배포하는 브랜치
2. 추가 데이터 import, 이전 모델 재학습, 기존 Endpoint로 모델 배포하는 브랜치

In [1]:
# General Setup
!pip install -q -Uqq kfp
!pip install -q -Uqq google-cloud-aiplatform
!pip install -q -Uqq google-cloud-pipeline-components

     |████████████████████████████████| 298 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 72.0 MB/s 
     |████████████████████████████████| 636 kB 66.0 MB/s 
     |████████████████████████████████| 106 kB 77.2 MB/s 
     |████████████████████████████████| 1.6 MB 60.6 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 6.9 MB/s 
     |████████████████████████████████| 10.9 MB 47.0 MB/s 
     |████████████████████████████████| 93 kB 1.9 MB/s 
     |████████████████████████████████| 75 kB 5.1 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [2]:
!gcloud init

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You must log in to continue. Would you like to log in (Y/n)?  Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapi

In [9]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.dsl import component

UnknownExtra: ignored

## prepare_cifar10_subsets

In [ ]:
# packages
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from random import sample
from shutil import copyfile, move

### Prepare Datasets and Cloud Function

- CIFAR10 dataset을 SPAN-1과 SPAN-2로 샘플링
- 지정된 버킷의 변경 사항을 모니터링하여 파이프라인을 트리거하는 Cloud Function 배포
- 트리거 되어야 하는 Cloud Function에 sampled dataset을 덤프

In [ ]:
# Download CIFAR10 Datasets(JPG)
!rm -rf CIFAR-10-images
!git clone https://github.com/YoongiKim/CIFAR-10-images.git

!rm -rf sampled

Cloning into 'CIFAR-10-images'...
remote: Enumerating objects: 60027, done.
remote: Total 60027 (delta 0), reused 0 (delta 0), pack-reused 60027
Receiving objects: 100% (60027/60027), 19.94 MiB | 49.20 MiB/s, done.
Resolving deltas: 100% (59990/59990), done.
Checking out files: 100% (60001/60001), done.


In [ ]:
# Labels and creating corresponding directories
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer']

span = 'span-1'

for label in labels:
  os.makedirs(f'sampled/{span}/{label}')

In [ ]:
# Sampling
tmp_dfs = []
num_samples = 200
bucket = 'my-cifar10-dataset-1012'

for label in labels:
  result = {}
  
  path = 'CIFAR-10-images/train/' + label + '/'
  imagefiles = [f'gs://{bucket}/{span}/{label}/{f}' for f in listdir(path) if isfile(join(path, f))]

  sampled = sample(imagefiles, num_samples)
  result['filename'] = sampled
  result['label'] = label

  tmp_dfs.append(pd.DataFrame.from_dict(result))

  for imagefile in sampled:
    imagefile = imagefile.split('/')[-1]
    move(f'{path}{imagefile}', f'sampled/{span}/{label}/{imagefile}')